# **THE STOLEN SZECHUAN SAUCE CASE colab**

<span style="color:red">**THIS IS THE UNSOLVED NOTEBOOK. YOU CAN FIND THE SOLVED NOTEBOOK HERE: https://github.com/testingvicc/testing/blob/main/SOLVED_TheStolenSzechuanSauceCase.ipynb**</span>

<img src="https://github.com/ds4n6/odsc_notebooks_colab/blob/main/Pictures/ODSC_JGv0.5.jpg?raw=1">


# **INSTALLING PACKAGES**

In [ ]:
!pip install ds4n6_libhttps://github.com/testingvicc/testing/blob/main/SOLVED_TheStolenSzechuanSauceCase.ipynb
!pip install qgrid==1.3.0
!pip install pandas_bokeh
!pip install untangle

# **IMPORTS**

In [ ]:
# Definition of variables & run imports
# Data Science Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# GUI Imports
import qgrid
from IPython.display import display, Markdown
#import ipynb_path

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import pandas_bokeh
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, output_file, show


# DS4N6
import ds4n6_lib.d4       as d4
import ds4n6_lib.common   as d4com
import ds4n6_lib.autoruns as d4atrs
import ds4n6_lib.fstl     as d4fstl
import ds4n6_lib.plaso    as d4pl
import ds4n6_lib.kansa    as d4ksa
import ds4n6_lib.kape     as d4kp
import ds4n6_lib.evtx     as d4evtx
import ds4n6_lib.mactime  as d4mctm
import ds4n6_lib.unx      as d4unx
import ds4n6_lib.utils    as d4utl
import ds4n6_lib.gui      as d4gui
import ds4n6_lib.ml       as d4ml
import ds4n6_lib.tshark   as d4tshrk
import ds4n6_lib.flist    as d4flst

from ds4n6_lib.common import analysis
from ds4n6_lib.common import anl
from ds4n6_lib.common import whatis
from ds4n6_lib.common import find_anomalies
from ds4n6_lib.unx    import xgrep
from ds4n6_lib.gui    import xread
from ds4n6_lib.gui    import xmenu
from ds4n6_lib.gui    import xdisplay
from ds4n6_lib.gui    import xanalysis

# Standard python imports
from time import sleep
import datetime
import time
import re
import os.path
import nbformat as nbf
import calendar
import glob
import xmltodict
import untangle
import importlib
import logging
import subprocess
import tarfile

# Data Save/Restore
import dill
import pickle
import warnings
import gdown

warnings.filterwarnings("ignore",category=DeprecationWarning)
warnings.filterwarnings("ignore",category=UserWarning)
warnings.filterwarnings("ignore")

#Tshark
import urllib

from google.colab import output
output.enable_custom_widget_manager()


pd.set_option('display.width', None) 
pd.set_option("max_columns", None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.colheader_justify', 'left')
pd.set_option("max_rows", 10)


In [ ]:
home_path=!pwd
thisnb=home_path[0]+"/ODSC_TheStolenSzechuanSauceCase_colab.ipynb"

Now, let's download the evidence already processed from Google drive:

In [ ]:
url = 'https://drive.google.com/uc?id=17OoXA5evLW6zef5gqu5rQoAvw8llUygY'
of = 'Data_ODSC.tgz'
gdown.download(url, of, quiet=False)
tar = tarfile.open("Data_ODSC.tgz")
tar.extractall()

# **FORENSIC EVIDENCE**

This section lists the evidence we will use in this case. 

**Evidence**, in cyber forensics, is information of probative value that is stored or transmitted in binary form. This means that once an incident is identified, this type of information is extracted to establish what has happened.

The evidence we will use for this case corresponds with the devices DC01 (Domain Controller) and Desktop: 
 - DC01 Disk Image (EO1)

 - DC01 Autoruns

 - Case001 PCAP

 - Desktop Disk Image (E01)

 - Desktop Autoruns

You can find more information at: https://dfirmadness.com/the-stolen-szechuan-sauce/

# **FORENSIC PROCESSING**

As previously mentioned, the **evidence** is acquired in binary form. Therefore, it is necessary to process and parse it, so that it can be understood and visualized by the forensic analyst.

There are different ways to process it, depending on the type of information one tool or another will be used.



**Plaso**

Plaso (Plaso Langar Að Safna Öllu), or super timeline of all the things, is a Python-based engine used by several tools for automatic creation of timelines. Plaso's default behavior is to create super timelines but it also supports creating more targeted timelines.

You can read more about it here: https://plaso.readthedocs.io/en/latest/

But, what is a **forensic artifact**? An artifact in this context is an operating system object which provides information about the activity or configuration of the computer or the users. Examples can be: Windows registry, Prefetch files, Memory objects...

Creating a Timeline with Plaso

---



 - **Using log2timeline and psort**:

    You can use “log2timeline” and “psort”. For example:

In [ ]:
# Do not run this cell. It is intended to be run on the console.

# log2timeline.py --storage-file timeline.plaso image.raw
# psort.py -o dynamic -w registrar.csv timeline.plaso

**Mactime**

Creates an ASCII timeline of file activity based on the output of the fls tool. It can be used to detect anomalous behavior and reconstruct events. The fls command must use the -m flag to generate an output with timestamps.

Mactime reads the body file (using the '-b' argument), which contains a line for each file or event. Mactime then sorts the data based on its temporal data and prints the result. It can optionally use a starting date or a date range to limit the data being printed. 

You can read more about it here: https://wiki.sleuthkit.org/index.php?title=Mactime

Creating a Timeline with Mactime


---



In [ ]:
# Do not run the cell. It is intended to be run on the console
# mactime -b body.txt 2002-03-01 > tl.03.01.2002.txt

Some of the arguments for mactime help to make the output more readable. On a Unix system, the User and Group IDs can be mapped to actual names by using the '-p' and '-q' flags. The '-z' flag can be used to specify the time zone, if it is different from the local timezone. 

In [ ]:
# Do not run the cell. It is intended to be run on the console
# mactime -b body.txt -z EST5EDT 2002-03-01 > tl.03.01.2002.txt

If you are going to include the resulting timeline in a document, then maybe it is better to supply the '-d' argument to output in comma delimited format. The resulting timeline can then be imported into a spread sheet and included as a table.

The '-i' option in 'mactime' creates an index summary file, including how many hits were found per day or hour. Using '-d' with '-i' allows one to easily import data into a spread sheet that can be graphed to spot suspicious behavior.

In [ ]:
# Do not run the cell. It is intended to be run on the console
# mactime -b body.txt -d -i hour data/tl-hour-sum.txt > timeline.txt

**Autoruns**

Autoruns is used to explore or detect persistent mechanisms. It reports Explorer shell extensions, toolbars, browser helper objects, Winlogon notifications, auto-start services, and much more. Autoruns goes way beyond other autostart utilities.

- Displays programs that are configured to run during bootup or login
- Lists the programs and drivers that will start from startup folders and registry keys



<img src="https://github.com/ds4n6/odsc_notebooks_colab/blob/main/Pictures/autoruns.jpg?raw=1">


**Tshark**

TShark is a network protocol analyzer. It lets you capture packet data from a live network, or read packets from a previously saved capture file, either printing a decoded form of those packets to the standard output or writing the packets to a file.

You can read more about it here: https://www.wireshark.org/docs/man-pages/tshark.html

Creating a Timeline with Tshark

---



Network packets are in timeline sequence from the earliest to the latest packets.

The -n option disables network object name resolution (such as hostname, TCP and UDP port names). The -r option reads the specified PCAP file.
The -Y option will filter based on Wireshark filters. The -T option displays the type of data (ek,fields,json and pdml) with -E/-e that controls the fields that you want.

In [ ]:
# Do not run the cell.
# tshark -n -r body.pcap -Y "http.request.method == "GET"" -T fields -Eheader=y -e ip.src -e ip.dst -e tcp.srcport -e tcp.dstport -e http.request.uri -e frame.time'

# **FORENSIC DATA READING**

## **READING FILES WITH PANDAS:**

After parsing the Disk Image evidence with plaso, we have obtained a JSON that we need to load into the notebook to be able to analyze it in detail.

JSON files can be read with pandas. All you need to do is use the **pd.read_json()** function, that will convert your JSON into a pandas object. You can find more information at https://pandas.pydata.org/pandas-docs/version/1.1.3/reference/api/pandas.read_json.html


In [ ]:
plaso_JSON = pd.read_json(home_path[0]+'/Data/DC_plaso.json')

Let's see how the JSON file looks like:

 - With the type() function we can see the object type that we obtained

In [ ]:
type(plaso_JSON)

 - Next, let’s see what its columns are and how many it has

In [ ]:
plaso_JSON.columns

 - Each column corresponds to a different event. Let’s select one of them and see what information it contains.

In [ ]:
plaso_JSON['event_0']

And what about other file types? We can read, for example, a Filesystem Timeline parsed with mactime in CSV format. These files can be read with **pd.read_csv()**. You can find more information at https://pandas.pydata.org/pandas-docs/version/1.1.3/reference/api/pandas.read_csv.html

**Filesystem Timeline**

 
This file (fstl) provides information on the system activity in a timeline. This will help us to see traces of malicious activity over time.

- Output from fls tool
- Lists the file and directory in a filesystem

You can find more information at: https://wiki.sleuthkit.org/index.php?title=Timelines

In [ ]:
mactime_CSV = pd.read_csv(home_path[0]+'/Data/DC_fstl.csv')

In [ ]:
mactime_CSV.columns

In [ ]:
type(mactime_CSV)

The CSV is saved in a DataFrame. We can see what its shape looks like without printing all the lines using the **head()** function, which gives us the first 5 lines.

In [ ]:
mactime_CSV.head()

## **READING FILES WITH CHRYSALIS**


Can we make it easier? We can read many data types with only one function from CHRYSALIS python library: **xread()**. This function not only reads the data and provides you with a DataFrame, like the previous ones we have seen with pandas, but it transforms and harmonizes the information to have it in the best possible way.

You can read more about it here: https://www.ds4n6.io/tools/chrysalis.html

Let's start reading a plaso file from Desktop. There are different fields that we can fill in to read in one way or another, but the required fields are only two: tool and rootpath. Tool refers to the tool used to parse evidence (plaso, mactime...) and rootpath refers to the path where the file is stored.

Once we run the cell we will see the file (if rootpath is correct) or we will have to look in the drop-down menu (if the rootpath is incorrect or empty).

<span style="color:purple">
    <b>INSTRUCTIONS</b>
    
<ul>
    <li>Execute cell</li>
    <li>Click SELECT & READ</li>
</ul>
    </span>

In [ ]:
xread(tool='plaso',rootpath=home_path[0]+'/Data/DESKTOP_plaso.json')

After the execution of the previous function, we can see all the artifacts extracted and harmonized (we will see later what harmonize actually means). This function saves the result in a variable called d4.out. So... Let's name another variable to save the information inside:

In [ ]:
pldfs_dsk = d4.out

As previously mentioned, reading isn't everything that this function does! It can **harmonize** the data and convert it into a **Dictionary of DataFrames**

**Nomenclature for variables in the DS4N6 project**
 - **[tool]df**: A single DataFrame that saves all the file information
 - **[tool]dfs**: A dictionary composed of DataFrames that save the file information divided by artifacts
 
For example, if we have a plaso we will create a dictionary and the variable name will be pldfs. On the other hand, if we use a fstl we will receive only one DataFrame and the variable name will be fstldf.

Let's see all the DataFrames names inside pldfs_dsk with **keys()** function, which allows us to see every key on the dictionary

In [ ]:
pldfs_dsk.keys()

Then let’s select one of the DataFrames and see how it looks like. After using **xread()** the information will be much more accesible and easier to read than in a JSON object. We won't have a column per event, but a row. Besides, we will have columns (those prefixed with D4_) with the aim of being explanatory columns, which will help us understanding the data.

In [ ]:
pldfs_dsk['windows_registry_installation']

Now let's read the plaso output from the Domain Controller following the same steps:

<span style="color:purple">
    <b>INSTRUCTIONS</b>
    
<ul>
    <li>Execute cell</li>
    <li>Click SELECT & READ</li>
</ul>
    </span>

In [ ]:
xread(tool='plaso',rootpath=home_path[0]+'/Data/DC_plaso.json')

In [ ]:
pldfs_srv = d4.out

Now we have stored in two different variables the plaso data of the two machines that we have to analyze:
 - **Domain Controller Server**: pldfs_srv
 - **Desktop**: pldfs_dsk
 
This information comes from the evidence of the disk image. Therefore, it will provide us with valuable information through a large number of different artifacts.

The next file we are going to read will be filesystem timeline. It will return us a single DataFrame with all the information inside it. In addition, this reading will be faster than the previous ones.

<span style="color:purple">
    <b>INSTRUCTIONS</b>
    
<ul>
    <li>Execute cell</li>
    <li>Click SELECT & READ</li>
</ul>
    </span>

In [ ]:
xread(tool='mactime',rootpath=home_path[0]+'/Data/DC_fstl.csv')

In [ ]:
fstldf = d4.out

And now, following the same steps, we will proceed to the reading of the autoruns file. In this case we need to specify its encoding, which being different from utf-18, not specifying it will give us an error. Therefore, in this **xread()** execution we will add a new option which will be encoding = 'utf-16'.

<span style="color:purple">
    <b>INSTRUCTIONS</b>
    
<ul>
    <li>Execute cell</li>
    <li>Click SELECT & READ</li>
</ul>
    </span>

In [ ]:
xread(tool='autoruns',rootpath=home_path[0]+'/Data/DC_autoruns.csv',encoding='utf-16')

In [ ]:
atrsdf = d4.out

# **INVESTIGATION**

Now that we have all the necessary information, we can start the investigation. Its objective will be identifying the attack and damage caused by the attacker.

## **1. OVERVIEW**

<span style="color:blue">**What are the hostnames for the Desktop and DC?** </span>

<span style="color:blue">**How many users are there on the Desktop?** </span>

### 1.2 Understanding of Evidence

The first thing that needs to be done on an investigation is understanding the general context of the electronic devices that provided us with the evidence. We can use the plaso output for that:

 - pldfs_dsk (Desktop)
 - pldfs_srv (Domain Controller/Server)

We will need to check:

 - Operating System
 - Installation Date
 - Timezone
 - Users

### 1.3 Using DataFrames to View Evidence

First, let’s see what DataFrames among all can help us to obtain this information. To do this, we will make a selection of them after viewing all with the function **keys()**

In [ ]:
pldfs_srv.keys()

The DataFrames we need to look at are:

 - **windows_registry_installation**: Which contains the information about the operating system
 - **windows_registry_timezone**: Which contains the information about the timezone
 - **windows_registry_sam_users**: Which contains the information about the users

 Let's start with the Domain Controller. We will select with pandas the DataFrame 'windows_registry_installation', which will give us information about the operating system. 

In [ ]:
pldfs_srv['windows_registry_installation']

In [ ]:
pldfs_dsk['windows_registry_installation']

### 1.4 Simple() Function

As a special tip, we can use **simple()** function from CHRYSALIS library to read the information better. This function hides the useless columns, doesn't repeat constant columns and simplifies the data.

Let's use it for reading the other DataFrames we need: 'windows_registry_timezone' and 'windows_registry_sam_users':

In [ ]:
pldfs_srv['windows_registry_timezone'].simple()

In [ ]:
pldfs_srv['windows_registry_sam_users'].simple()

But... Can we make it even better? With CHRYSALIS we can select analysis functions from a menu! We only need to call **xanalysis()** function. Let's see how it works.

<span style="color:purple">
<b>INSTRUCTIONS:</b>

<ul>
    <li>Execute the cell</li>
    <li>Select Dictionary as analysis object</li>
    <li>Select plaso for the type of analysis</li>
    <li>Select Overview as the available analysis</li>
</ul>
</span>


In [ ]:
xanalysis(pldfs_srv)

As we can see, with this function only the relevant information of the different DataFrames of interest is shown. Let's repeat it for Desktop this time.

In [ ]:
xanalysis(pldfs_dsk)

<span style="color:blue">**What are the hostnames for the Desktop and DC?** </span>

**ANSWER HERE**:


<span style="color:blue">**How many users are there on the Desktop?** </span>

**ANSWER HERE**:


### **1.5 CONCLUSIONS**:

**Overview**
 - On the Domain Controller: 
     - The operating system is Windows Server 2012 R2 Standard Evaluation
     - The Timezone is Pacific Standard Time
     
     
 - On the Desktop:
     - The operating system is Windows 10 Enterprise Evaluation
     - The Timezone is Pacific Standard Time

## **2. SUCCESSFUL LOGON ANALYSIS**

The next step of an attack is checking the logon information. We need to check who had access to the system.

As mentioned before, an artifact is data that can contain a trace of malicious activity. In order to check the accesses to the two devices, we will use a plaso artifact: The Windows evtx files. 

<span style="color:blue">**Which are the events that give us logon information?** </span>

<span style="color:blue">**Is there any suspicious logon on the Domain Controller/Server?** </span>

<span style="color:blue">**Which logon type is it and what does it mean?** </span>

<span style="color:blue">**What about the Desktop?** </span>

### 2.1 Windows Events

**Windows evtx Files**
-
EVTX is a file extension commonly associated with Windows 7+ Event Log Format files. Windows 7+ Event Log Format specification was created by Microsoft. EVTX files are supported by software applications available for devices running Windows. Files with EVTX extension are categorized as System Files files. They contain useful information about system and users such as authentication, file changes, commands executed, etc.

There are different types of Windows events. In this case we will use the following Security events ID

### 2.2 Windows Security Events


**Security evtx**
-
Contains events such as: 
- Authentications (valid and invalid logon attempts (Event ID 4624 and 4625 respectively))

    - **4624**: Successful logon
    - **4625**: Failed logon
    
    
- Objects access (Creation/Modification/Deletion of files)

As we know, we have a plaso dictionary that contains quite a few DataFrames and one of them is 'windows_evtx_record'. The next step is converting that DataFrame of events into a dictionary that, instead of containing all the events information together, has different DataFrames for the different categories of events (Security.evtx, System.evtx, etc.)

In [ ]:
pldfs_dsk.keys()

<span style="color:blue">**Which are the events that give us logon information?** </span>

**ANSWER HERE**:


### 2.3 plaso_get_evtxdfs() Function


For that purpose, CHRYSALIS has **plaso_get_evtxdfs()** function which only needs the plaso dictionary name and the hostname that we saw on the Overview analysis.

In [ ]:
evtxrawdfs_dsk = d4pl.plaso_get_evtxdfs(pldfs_dsk, 'DESKTOP-SDN1RPT')

Now that the dictionary is created, let's see how it looks using **keys()** function.

In [ ]:
evtxrawdfs_dsk.keys()

As previously mentioned, we want Security.evtx and we are particularly interested on its 4624 and 4625 event IDs.

For that reason, we are going to select 'Security.evtx' from the 'evtxrawdfs_dsk' dictionary, resulting in a DataFrame: 'secevtxrawdf_dsk'. This DataFrame contains the raw Security events information, in other words, all events with different IDs.

In [ ]:
secevtxrawdf_dsk=evtxrawdfs_dsk['Security.evtx']
secevtxrawdf_dsk

Now that we obtained a DataFrame with only Security events, wouldn’t it be even better to distinguish by ID? We can create a dictionary of events separating them by ID with **evtid_dfs_build()** function from CHRYSALIS. Let's see how it works!

In [ ]:
secevtxdfs_dsk = d4evtx.evtid_dfs_build(secevtxrawdf_dsk)

Finally, we can create a DataFrame with only 4624 (Successful logon attempts) events.

In [ ]:
evts4624df_dsk = secevtxdfs_dsk[4624]

We can repeat the same steps for the Domain Controller.

In [ ]:
evtxrawdfs_srv = d4pl.plaso_get_evtxdfs(pldfs_srv, 'CITADEL-DC01')

In [ ]:
# Create Raw DF with plaso evtx records
secevtxrawdf_srv=evtxrawdfs_srv['Security.evtx']
# Create DF dict with individual Event IDs
secevtxdfs_srv = d4evtx.evtid_dfs_build(secevtxrawdf_srv)
evts4624df_srv = secevtxdfs_srv[4624]

Now's when the magic happens!

With **xanalysis()** function from CHRYSALIS we can see a statistical analysis of events 4624 and, consequently, obtain valuable information regarding the accesses to the machine.

How does it work? As explained before, you have to select the options you want from the drop-down menus. In this case we must select only the type of analysis, which in this case will be "Logon info summary", since we have a DataFrame ("evts4624df_srv") and we do not have the option of Dictionary.

If you want to explore different options you can load the event dictionary by ID "secevtxdfs_srv".

### 2.4 xanalysis() Function

<span style="color:purple">
<b>INSTRUCTIONS:</b>
    <ul>
    <li>Execute the cell.</li>
    <li>Select Logon info summary as the available analysis.</li>
    <li>Click on Export Result to d4.out.</li>
</ul>
</span>

**ADVICE: If you see the graph very small execute the function again.**

In [ ]:
# Domain Controller
xanalysis(evts4624df_srv)

As we can see, there's a lot of information to process here. So let's divide it in different sections:
 - **Workstation name table**: This is also known as the machine name or host name. It is the unique name you or your system administrator gave to the machine you are using. This is usually not the same name as your user name or login name. This is the machine name that other users see over the network, such as from electronic mail. The information in this table represents all the workstation names that appear in the data and how many times they appear. In this way we can see if they are common or exceptional.
 - **IP ADDRESS table**: An IP address is a unique address that identifies a device on the internet or a local network. IP stands for "Internet Protocol," which is the set of rules governing the format of data sent via the internet or local network. The information in this table represents all the IP addresses that appear in the data and how many times they appear. This way we can see, again, if they are common or exceptional.
 - **FIRST table**: The Target User Name column indicates the account for whom the new logon was created, i.e. the account that was logged on. On the Timestamp column we can see the first time that the user logged on. This gives us valuable information because we can actually see if its a new user or a common one.
 - **LAST table**: In this case the information provided is the same as in the previous table but with the last time that the user logged on.
 - **STATS table**: The information in this table represents all the Target User Names that appear in the data and how many times they appear. In this way we can see if they are common or exceptional.
 - **SID table**: A SID is a number used to identify user, group, and computer accounts in Windows. They're created when the account is first made in Windows and no two SIDs on a computer are ever the same. This table shows us the user name matching its SID.

- **Logons graph:** This graph shows the different accesses that a combination of Workstation Name, User name, Ip Address and LogonType has had. But, first things first... How many logon types we have? What does this mean?

    *   2: Interactive (logon at keyboard and screen of system)
    *   3: Network (i.e. connection to shared folder on this computer from elsewhere on network)
    *   7: Unlock (i.e. unnattended workstation with password protected screen saver)
    *   10: RemoteInteractive (Terminal Services, Remote Desktop or Remote Assistance)


This graph offers us the possibility of identifying patterns visually and, therefore, that we do not overlook something anomalous. Besides, we can also evaluate if the logon was dangerous or not.


 - **Logons table**: The last table contains the same information as the graph but on a DataFrame format. This way we can take a closer look and export it for making other analysis on our own. So let's export it and save it in a variable named "successful_logons_df"

In [ ]:
successful_logons_df = d4.out

In [ ]:
successful_logons_df.simple()

We can't read all rows because of the current configuration. There's a way of changing it manually and set all the display options one by one. In this case we would need **pd.options.display.max_rows()** function from pandas.

We can change other things like min rows, max columns, etc. To change all of them at the same time we use **xdisplay()** function from CHRYSALIS. In this case, we want more rows, so we need to change "display.max_rows" for 25. Let's see how it works!

<span style="color:purple">
<b>INSTRUCTIONS:</b>
    <ul>
    <li>Execute the cell.</li>
    <li>On 'display:max_rows' change the number for 25.</li>
</ul>
</span>


In [ ]:
xdisplay()

In [ ]:
successful_logons_df.simple()

Let's talk about everything we discovered on the analysis. We can see at first view that there's a very strange Workstation Name: kali. Besides, the "194.61.24.102" IP address catches our attention because it's not a common one on the machine. Finally, on "successful_logons_df" rows 18 and 20 have never happened before. We can clearly see it on the graph.

**So let's keep in mind kali and 194.61.24.102**

<span style="color:blue">**Is there any suspicious logon on the Domain Controller/Server?** </span>

**ANSWER HERE**:


<span style="color:blue">**Which logon type is it and what does it mean?** </span>

**ANSWER HERE**:


We can repeat the same steps for the Desktop:

<span style="color:purple">
<b>INSTRUCTIONS:</b>
    <ul>
    <li>Execute the cell.</li>
    <li>Select Logon info summary as the available analysis.</li>
</ul>
</span>

**ADVICE: If you see the graph very small execute the function again.**

In [ ]:
xanalysis(evts4624df_dsk)

We can see that 7 and 9 have never happened before and that's suspicious.

**Let's not forget 10.42.85.10 IP address**

<span style="color:blue">**What about the Desktop?** </span>

**ANSWER HERE**:


### **2.5 CONCLUSIONS**:

**2. Logon Analysis**

 - On the Domain Controller: Suspicious activity on the 19th of September 2020 (kali and 194.61.24.102 -> User administrator)
 - On the Desktop: Suspicious activity on the 19th of September 2020 (User administrator)

## **3. CLOSER LOOK INTO DOMAIN CONTROLLER LOGONS**

<span style="color:blue">**At what time did the suspicious logon ocurred on the Domain Controller/Server?** </span>

### 3.1 Suspicious Logons

Let's look at the suspicious logons closer, reading the 4624 DataFrames from Domain Controller directly from the DataFrame, using the **simple()** function from CHRYSALIS again.

In [ ]:
secevtxdfs_srv[4624].simple()

There are 2540 rows so it would be a huge DataFrame using **xdisplay()**... Why don't we use **xmenu()**?

**xmenu()** is a function from CHRYSALIS which allows us to display huge DataFrames, select them with a menu directly from a dictionary, apply the simple function and beautify them. 

<span style="color:purple">
<b>INSTRUCTIONS:</b>

<ul>
    <li>Execute the cell</li>
    <li>Select 4624 as DataFrame on the left drop-down menu</li>
    <li>Select qgrid as grid on the middle drop-down menu<\li>
    <li>Search for the logons we previously saw on the logon analysis. Pay special attention to the Timestamp, WorkstationName and IpAddress columns<\li>   
</ul>
</span>
Let's see how it works!

In [ ]:
xmenu(secevtxdfs_srv)

We selected DataFrame 4624 and qgrid. This way we can display all the information and filter it by columns.

<span style="color:blue">**At what time did the suspicious logon ocurr on the Domain Controller/Server?** </span>

**ANSWER HERE**:




## 3.2. Checking Failed Logons

The next question would be: How did the atacker manage to enter? The most usual cases are Brute Force (making a lot of attempts to find the correct password) and credentials theft. We can check the 4625 events (that had failed logons) to get more information and check if it's been Brute Force.

<span style="color:purple">
<b>INSTRUCTIONS:</b>

<ul>
    <li>Execute the cell</li>
    <li>Select 4625 as DataFrame on the left drop-down menu</li>
    <li>Select qgrid as grid on the middle drop-down menu<\li>
</ul>
</span>

In [ ]:
xmenu(secevtxdfs_srv)

There are many failed logons from **kali**. This seems a Brute Force network.

### **3.3 CONCLUSIONS**:

**3. CLOSER LOOK INTO DOMAIN CONTROLLER LOGONS**

 - Successful logon at 03:21 AM from kali
 - Many failed logons from kali trying to enter on the Domain Controller

## **4. FAILED LOGONS**

<span style="color:blue">**How many failed logons occurred on the Domain Controller/Server?** </span>


<span style="color:blue">**At what time was the Brute Force used?** </span>


Now that we’ve seen the data, let’s create a graph that shows us what happened in a more visual way.

### 4.1. **Matplotlib**

Matplotlib is a comprehensive library for creating static, animated, and interactive visualizations in Python.

- Allows you to plot a DataFrame.
- Allows editing many aspects of the chart.
- There are many types of graphics from which you can choose.


**Plotting the failed logon events based on the hourly timestamps can help us understand if there's been Brute Force**


First, we’re going to create a DataFrame that contains 4625 events and we’re going to select only the EventID_ column. In addition, we will use the **head()** function to visualize its first 5 lines and to understand how it is.

In [ ]:
failedlogons = secevtxdfs_srv[4625][['EventID_']]
failedlogons.head()

Next, we will use the index, which contains the date, to create a new column.

In [ ]:
failedlogons['Timestamp'] = failedlogons.index
failedlogons.head()

Next, let’s do an hourly count of the number of times the event 4625 appears on the DataFrame.

In [ ]:
unique_logons_df = pd.Series(failedlogons['Timestamp']).resample('H').nunique()
unique_logons_df.head()

And finally, we’ll make a plot of the DataFrame that we created with the Matplotlib library. It is as simple as writing the name of the DataFrame and then using the function **.plot.bar()**, which will show us a bar graph.

In [ ]:
unique_logons_df.plot.bar()

**In this way, we can easily see that there has been a large number of failed logon attempts on the computer without using the DataFrame and searching line by line.**

## 4.2. Failed Logons Analysis

As on other occasions, we can do it even more easily, using the **xanalysis()** function of the CHRYSALIS library. 

<span style="color:purple">
<b>INSTRUCTIONS:</b>

<ul>
    <li>Execute the cell</li>
    <li>Select DataFrame as analysis object</li>
    <li>Select evtx for the type of analysis</li>
    <li>Select 4625 as DF to analyze </li>
    <li>Select Failed Logons info as the available analysis</li>
</ul>
</span>


In [ ]:
xanalysis(secevtxdfs_srv)

<span style="color:blue">**How many failed logons occurred on the Domain Controller/Server?** </span>


**ANSWER HERE**:


<span style="color:blue">**At what time was the Brute Force used?** </span>


**ANSWER HERE**:


### **4.3. CONCLUSIONS**:

**5. FAILED LOGONS**

 - There's been Brute force on the 19th of September 2020

## **5. LOOKING INTO THE FSTL & AUTORUNS**

<span style="color:blue">**Is there any suspicious activity?** </span>


<span style="color:blue">**Was there an Internet download?** </span>

<span style="color:blue">**When was the Szechuan Sauce modified?**</span>

<span style="color:blue">**Is coreupdater.exe a malware?** </span>

### 5.1 Filesystem Timeline

Once we have verified that there has been an attacker’s logon, the main thing is to understand what damage or activity has been done on the computer. To do this, we will use the Filesystem Timeline, which as previously mentioned in the notebook, provides information about the system’s activity on a timeline.

We will filter by date using **.loc()** from pandas. This will allow us to see the activity from the moment of the attack. Let's see which column we need to use!

In [ ]:
fstldf.columns

Let's select the date 2020-09-18 22:00:00 from the Date column to see what happened from the previous day.

<span style="color:purple">
    <b>INSTRUCTIONS</b>
    
<ul>
    <li>Execute the cell</li>
    <li>Select '+ DataFrame' as DataFrame on the left drop-down menu</li>
    <li>Select qgrid as grid on the middle drop-down menu</li>
</ul>
</span>

In [ ]:
fstldf_DC_filterbydate = fstldf.loc[(fstldf['Date'] >= '2020-09-18 22:00:00')]
xmenu(fstldf_DC_filterbydate)

To understand the FSTL correctly, we will see that there is a column called MACB, this column gives us information about the actions performed on a file. To interpret it, depending on the letters that appear (m, a, c or b) we can get help from the following **SANS** table:

<img src="https://github.com/ds4n6/odsc_notebooks_colab/blob/main/Pictures/MACB.webp?raw=1">


<span style="color:blue">**Is there any suspicious activity?** </span>


**ANSWER HERE**:


<span style="color:blue">**Was there an Internet download?** </span>

**ANSWER HERE**:


<span style="color:blue">**When was the Szechuan Sauce modified?** </span>

**ANSWER HERE**:


### 5.2 Autoruns

We can look for it on autoruns. In this way, we will check if it was moved on the Services and we will be able to see its hash. To locate coreupdater.exe, we will use the **xgrep()** function of the CHRYSALIS library, which allows us to search for a string within any column or row of the DataFrame.

In [ ]:
xgrep(atrsdf, 'coreupdater')

**It's been installed with persistence and we can obtain the hash from the SHA-256 column: 10F3B92002BB98467334161CF85D0B1730851F9256F83C27DB125E9A0C1CFDA6**

### 5.3 VirusTotal

To check what coreupdater.exe is, we can use **VirusTotal**. It analyzes suspicious files and URLs to detect types of malware and automatically shares them with the security community. We can search for coreupdater hash (10F3B92002BB98467334161CF85D0B1730851F9256F83C27DB125E9A0C1CFDA6) here: https://www.virustotal.com/gui/home/search

<img src="https://github.com/ds4n6/odsc_notebooks_colab/blob/main/Pictures/virustotal.png?raw=1">

<span style="color:blue">**Is coreupdater.exe a malware?** </span>

**ANSWER HERE**:


## **6. GETTING MORE INFORMATION FROM SHELLBAGS**

What can we do next? We can use the Shellbag to check what folders the atacker visited.

<span style="color:blue">**Which folders did the atacker visit?** </span>


### **6.1 Shellbags**

- Set of registry keys that store details about a viewed folder, such as its size, position, and icon.
- It helps track views, sizes and positions of a folder window when viewed through Windows Explorer; this includes network folders and removable devices.

To select the DataFrame we are interested in, we will use **xmenu()** from CHRYSALIS. 

<span style="color:purple">
    <b>INSTRUCTIONS</b>
    
<ul>
    <li>Execute the cell</li>
    <li>Select 'windows_shell_item_file_entry' as DataFrame on the left drop-down menu</li>
    <li>Select qgrid as grid on the middle drop-down menu</li>
</ul>
</span>


In [ ]:
xmenu(pldfs_srv)

We are particularly intrigued by a 'Secret' file in the 'Fileshare' directory and the access to the Downloads folder, probably after coreupdater.exe download.

<span style="color:blue">**Which folders did the atacker visit?** </span>

**ANSWER HERE**:


### **6.2. CONCLUSIONS**:

**6. GETTING MORE INFORMATION FROM SHELLBAGS**

The folders the attacker visited are:
- 'Secret' file on Fileshare
- There's been an access to the Downloads folder 

## 7. Analyzing Network Traffic

<span style="color:blue">**What is the port number that the attacker downloaded the malware from?** </span>


### 7.1. TShark

TShark is a network protocol analyzer. 
It allows you to:
- Capture packet data from a live network
- Read packets from a previously saved capture file
- Output and write the packets to a file. (JSON, CSV etc.)

In this demo, we will make use of a generated CSV files using different Wireshark filters on the PCAP provided by the case.


### 7.2. HTTP

The following command was used with Tshark to find out about the HTTP GET Method:

tshark -n -r <pcap location> -Y "http.request.method == "GET"" -T fields -Eheader=y -e ip.src -e ip.dst -e tcp.srcport -e tcp.dstport -e http.request.uri -e frame.time
    
HTTP GET Request Method was used to help us filter the packets to help us find out how was coreupdater.exe introduced into the victims' systems.
    
<span style="color:purple">
    <b>INSTRUCTIONS</b>
    
<ul>
    <li>Execute cell</li>
    <li>Click SELECT & READ</li>
</ul>
    </span>

In [ ]:
xread(tool='tshark', rootpath=home_path[0]+'/Data/PCAP_http.pcap.csv')

In [ ]:
pcaphttpdf = d4.out

We shall make use of **xgrep()** from CHRYSALIS to find out how coreupdater.exe was downloaded

In [ ]:
xgrep(pcaphttpdf, 'coreupdater')

**NOTE: Packets are in CEST and Domain Controller was in Pacific Standard Time, hence we have to minus 1 hour from packet time**


The IP address '194.61.24.102' hosted the malware file on port 80 (HTTP) that the victims were made to download onto their respective systems)
But how did the malicious IP know about the network?
Attackers usually performs reconnaissance to understand about the network and try to exploit exploit vulnerabilities.


### 7.3. Reconnaissance Scan

Let's replace the "http.request.method == GET" filter with "ip.addr == 194.61.24.102".
This filter helps us to understand about the malicious IP as both source and destination IP to learn what was done in the network.

<span style="color:purple">
    <b>INSTRUCTIONS</b>
    
<ul>
    <li>Execute cell</li>
    <li>Click SELECT & READ</li>
</ul>
    </span>

In [ ]:
xread(tool='tshark', rootpath=home_path[0]+'/Data/PCAP_nmap.pcap.csv')

In [ ]:
pcapnmapdf = d4.out

As there are many packets with the malicious IP, we can make use of the first few packets to learn about the behaviour using pandas' head()

In [ ]:
pcapnmapdf.head(20)

The very first packet, the malicious IP did a ping request Domain Controller (10.42.85.10). Ping/ICMP scan is one of the common methods to find out what devices are connected to the network.

Next few packets, within the very same second (04:19:13), there were two probes to port 443 (HTTPS) and 80 (HTTP) before getting ICMP reply from the Domain Controller.

This means that there was a service port scan done on domain controller itself.

### 7.4. Remote Desktop Protocol

About 13 seconds later at 04:19:26, the malicious IP is trying to connect to port 3389 (RDP) with incremental source TCP port number of 2 (38088, 38090, 38092, etc.)

This behaviour could mean that the malicious IP is trying to conduct a brute force attack to login with RDP.

Let's replace the filter with "ip.addr == 194.61.24.102 and tcp.dstport == 3389 and tcp.flags.syn==1"

As we know the malicious IP and the destination is port 3389, TCP SYN has to be 1 as it is always the start of TCP handshake.

To learn about TCP handshake, visit: https://www.guru99.com/tcp-3-way-handshake.html

<span style="color:purple">
    <b>INSTRUCTIONS</b>
    
<ul>
    <li>Execute cell</li>
    <li>Click SELECT & READ</li>
</ul>
    </span>

In [ ]:
xread(tool='tshark', rootpath=home_path[0]+'/Data/PCAP_rdp.pcap.csv')

In [ ]:
pcaprdpdf = d4.out

<span style="color:purple">
    <b>INSTRUCTIONS</b>
    
<ul>
    <li>Execute the cell</li>
    <li>Select '+ DataFrame' as DataFrame on the left drop-down menu</li>
    <li>Select qgrid as grid on the middle drop-down menu</li>
</ul>
</span>


In [ ]:
xmenu(pcaprdpdf)

We can see from **xmenu()** with the dataframe, that there were twenty thousands packets in just 2 minutes!
However, at 04:22:36 to 04:56:03, there were no other SYN packets.
This meant that the attacker successfully logon to Domain Controller where it downloaded the malware at 04:24:04

<span style="color:blue">**What is the port number that the attacker downloaded the malware from?** </span>


**ANSWER HERE**:


### **7.5. CONCLUSIONS**:

**7. Looking Into the PCAP File**

- The victims downloaded the malware from 194.61.24.102 using HTTP GET Method
- The attack hosted malware on a HTTP server